# 1. Factors

In [1]:
%run '_preamble.ipynb'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
available imports:
  import os
  import logging
  import pandas as pd
  import numpy as np

connect to this kernel with:
  jupyter console --existing 8b59cbb9-caff-4d5e-9649-41cd060269db

Logging to : "/Users/melle/software-development/thomas-master/logs/1. Factors.log"


In [2]:
from thomas.core import Factor

from IPython.display import display, HTML

In [3]:
# Create the Factors for the Sprinkler network. 
# Note that the order in which the states are defined is important. 
# As such, this will only work in Python ≥ 3.6!          ^^^^^^^^^

# P(A)
fA = Factor(
    [0.6, 0.4], 
    {'A': ['a1', 'a0']}
)

# P(B|A)
fB_A = Factor(
    [0.2, 0.8, 0.75, 0.25], 
    {'A': ['a1', 'a0'],'B': ['b1', 'b0']}
)

# P(C|A)
fC_A = Factor(
    [0.8, 0.2, 0.1, 0.9], 
    {'A': ['a1', 'a0'],'C': ['c1', 'c0']}
)

# Define a factor that holds the *conditional* distribution P(D|BC)
fD_BC = Factor(
    [0.95, 0.05, 0.9, 0.1,0.8, 0.2, 0.0, 1.0], 
    {'B': ['b1', 'b0'],'C': ['c1', 'c0'],'D': ['d1', 'd0']}
)

# P(E|C)
fE_C = Factor(
    [0.7, 0.3, 0.0, 1.0], 
    {'C': ['c1', 'c0'],'E': ['e1', 'e0']}
)

In [4]:
# fA is essentially a prior distribution.
fA

factor(A)
A
a1    0.6
a0    0.4
dtype: float64

In [5]:
# fB_A is holds a conditional distribution: P(B|A)
fB_A

factor(A,B)
A   B 
a1  b1    0.20
    b0    0.80
a0  b1    0.75
    b0    0.25
dtype: float64

In [6]:
# Multiplying the factor with a *prior* with a *conditional* distribution, yields
# a *joint* distribution.
fAB = fA * fB_A
display(fAB)

factor(A,B)
A   B 
a0  b0    0.10
    b1    0.30
a1  b0    0.48
    b1    0.12
dtype: float64

In [7]:
# By summing out A, we'll get the prior over B
fAB.sum_out('A')

factor(B)
B
b0    0.58
b1    0.42
dtype: float64

In [8]:
# To get to this prior directly, we could also have taken the dot (matrix) product,
# but we'd have to turn B_A into a matrix.
fA.as_series().dot(fB_A.unstack())

B
b0    0.58
b1    0.42
dtype: float64

In [9]:
# Order matters in matrix multiplication: this will fail (although a better error message might
# be helpful).
try:
    fB_A.as_series().dot(fA.as_series())
except Exception as e:
    print(e)

matrices are not aligned


In [10]:
# Unstacking the conditional distribution makes it easy to see that the rows sum to 1.
fB_A.unstack()

B     b0    b1
A             
a0  0.25  0.75
a1  0.80  0.20